In [31]:
!pip install langgraph

   ---------------------------------------- 0.0/153.7 kB ? eta -:--:--
   ------------------ --------------------- 71.7/153.7 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 153.7/153.7 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.4 kB ? eta -:--:--
   ---------------------------------------- 45.4/45.4 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/74.7 kB ? eta -:--:--
   -------------------------------- ------- 61.4/74.7 kB 3.4 MB/s eta 0:00:01
   -------------------------------- ------- 61.4/74.7 kB 3.4 MB/s eta 0:00:01
   -------------------------------- ------- 61.4/74.7 kB 3.4 MB/s eta 0:00:01
   -------------------------------- ------- 61.4/74.7 kB 3.4 MB/s eta 0:00:01
   -------------------------------- ------- 61.4/74.7 kB 3.4 MB/s eta 0:00:01
   -------------------------------- ------- 61.4/74.7 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 74.7/74.7 kB 229.3 kB/s eta 0:00:00


In [27]:
!pip install langchain openai
!pip install python-arango


In [10]:
!pip install langchain-community


   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
    --------------------------------------- 0.0/2.5 MB 991.0 kB/s eta 0:00:03
   - -------------------------------------- 0.1/2.5 MB 1.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.2/2.5 MB 1.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/2.5 MB 1.2 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.5 MB 1.3 MB/s eta 0:00:02
   ----- ---------------------------------- 0.4/2.5 MB 1.4 MB/s eta 0:00:02
   ------- -------------------------------- 0.5/2.5 MB 1.5 MB/s eta 0:00:02
   -------- ------------------------------- 0.5/2.5 MB 1.5 MB/s eta 0:00:02
   --------- ------------------------------ 0.6/2.5 MB 1.5 MB/s eta 0:00:02
   ----------- ---------------------------- 0.7/2.5 MB 1.5 MB/s eta 0:00:02
   ------------ --------------------------- 0.8/2.5 MB 1.5 MB/s eta 0:00:02
   ------------- -------------------------- 0.9/2.5 MB 1.6 MB/s eta 0:00:02
   --------------

In [54]:
!pip install langchain_openai
!pip install --upgrade langchain langchain-community langchain-openai langgraph

In [56]:
import os
from dotenv import load_dotenv
from arango import ArangoClient

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool

In [61]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

db = ArangoClient(hosts="http://localhost:8529").db('NeuThera', username='root', password='openSesame')

arango_graph = ArangoGraph(db)

In [63]:
arango_graph.schema

{'Graph Schema': [{'graph_name': 'NeuThera',
   'edge_definitions': [{'edge_collection': 'disease-disease',
     'from_vertex_collections': ['disease'],
     'to_vertex_collections': ['disease']},
    {'edge_collection': 'disease-drug',
     'from_vertex_collections': ['disease'],
     'to_vertex_collections': ['drug']},
    {'edge_collection': 'disease-function',
     'from_vertex_collections': ['disease'],
     'to_vertex_collections': ['go']},
    {'edge_collection': 'disease-gene',
     'from_vertex_collections': ['disease'],
     'to_vertex_collections': ['gene']},
    {'edge_collection': 'drug-drug',
     'from_vertex_collections': ['drug'],
     'to_vertex_collections': ['drug']},
    {'edge_collection': 'drug-gene',
     'from_vertex_collections': ['drug'],
     'to_vertex_collections': ['gene']},
    {'edge_collection': 'function-function',
     'from_vertex_collections': ['go'],
     'to_vertex_collections': ['go']},
    {'edge_collection': 'gene-function',
     'from_vertex_

In [ ]:
@tool
def text_to_aql_to_text(query: str):
    """This tool is available to invoke the
    ArangoGraphQAChain object, which enables you to
    translate a Natural Language Query into AQL, execute
    the query, and translate the result back into Natural Language.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    chain = ArangoGraphQAChain.from_llm(
    	llm=llm,
    	graph=arango_graph,
    	verbose=True,
        allow_dangerous_requests=True
    )
    
    result = chain.invoke(query)

    return str(result["result"])

In [66]:
tools = [ text_to_aql_to_text ]

def query_graph(query):
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
    app = create_react_agent(llm, tools)    
    final_state = app.invoke({"messages": [{"role": "user", "content": query}]})
    return final_state["messages"][-1].content

In [75]:
query_graph("Find me all drugs related to disease id MESH:D005923")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH disease, drug, disease-drug
FOR v, e IN 1..1 OUTBOUND 'disease/MESH:D005923' disease-drug
RETURN v

AQL Query Execution Error: 
syntax error, unexpected - operator near '-drug
FOR v, e IN 1..1 OUTBOUND ...' at position 2:28

AQL Query (2):
WITH disease, drug, `disease-drug`
FOR v, e IN 1..1 OUTBOUND 'disease/MESH:D005923' `disease-drug`
RETURN v

AQL Result:
[{'_key': 'DB00572', '_id': 'drug/DB00572', '_rev': '_jRhJSoC---', 'accession': 'APRD00807 | EXPT02427', 'drug_name': 'Atropine', 'cas': '51-55-8', 'unii': '7C0697DR9I', 'synonym': '(±)-atropine | (±)-hyoscyamine | Atropin | Atropina | Atropine | Atropinum | dl-Hyoscyamine | dl-tropyltropate | Tropine tropate', 'key': 'RKUNBYITZUJHSG-SPUOUPEWSA-N', 'chembl': 'CHEMBL517712', 'smiles': 'CN1[C@@H]2CC[C@H]1C[C@@H](OC(=O)C(CO)c1ccccc1)C2', 'inchi': 'InChI=1S/C17H23NO3/c1-18-13-7-8-14(18)10-15(9-13)21-17(20)16(11-19)12-5-3-2-4-6-12/h2-6,13-16,19H,7-11H2,1H3/t13-,14+,15+,16?

"The drugs related to the disease with the ID MESH:D005923 include:\n\n1. **Atropine** - Used for treating certain types of nerve agent and pesticide poisonings, and some types of slow heart rate.\n2. **Glutathione** - Known for its antioxidant properties and treatment of conditions related to oxidative stress.\n3. **Caffeine** - A central nervous system stimulant.\n4. **Amiloride** - Used to treat high blood pressure and heart failure.\n5. **Genistein** - Studied for potential effects on cancer and other health conditions.\n6. **LY-294002** - Used in research to inhibit certain cellular pathways.\n7. **Ciprofloxacin** - An antibiotic for various bacterial infections.\n8. **Beclomethasone dipropionate** - A corticosteroid for asthma and allergic rhinitis.\n9. **Tolcapone** - Used in managing Parkinson's disease.\n\nThese drugs are associated with the specified disease through established disease-drug relationships in the database."

In [49]:
llm = ChatOpenAI(model="gpt-4", temperature=0.2, openai_api_key=openai_api_key)


# Define a prompt template for AQL query generation based on the user's schema
aql_prompt = PromptTemplate(
    input_variables=["user_query"],
    template="""
    Given the user's query: "{user_query}", generate an AQL query that retrieves relevant proteins (ensp)
    from the `GP` (Gene-Protein) collection in ArangoDB.
    
    The query should:
    1. Find relevant genes in the `Gene` collection based on user input.
    2. Use the `GP` edge collection to retrieve proteins (ensp) linked to those genes.
    
    Example AQL Query:
    ```
    FOR g IN Gene
        FILTER CONTAINS(LOWER(g.gene_name), LOWER("{user_query}")) 
        FOR gp IN GP
            FILTER gp.ensg == g.ensg
            RETURN gp.ensp
    ```
    Ensure the query is properly formatted and follows the schema strictly.
    """
)


In [51]:
def generate_aql_query(data):
    user_query = data["user_input"]
    messages = [SystemMessage(content="You are an AI that generates valid AQL queries."), HumanMessage(content=user_query)]
    
    # Generate AQL query using OpenAI
    aql_query = llm.invoke(messages).content
    return {"aql_query": aql_query}

In [45]:
def execute_aql_query(data):
    aql_query = data["aql_query"]
    print("Generated AQL Query:", aql_query)  # Debugging
    
    try:
        # Execute the query using `python-arango`
        cursor = db.aql.execute(aql_query)
        results = [doc for doc in cursor]
    except Exception as e:
        print(f"Error executing AQL query: {e}")
        results = []

    return {"proteins": results}

In [46]:
from langgraph.graph import StateGraph
builder = StateGraph(dict)  # Corrected to StateGraph
builder.add_node("generate_query", generate_aql_query)
builder.add_node("execute_query", execute_aql_query)

# Define workflow connections
builder.set_entry_point("generate_query")
builder.add_edge("generate_query", "execute_query")

# Compile the LangGraph
graph = builder.compile()

In [47]:
def query_proteins(user_input):
    output = graph.invoke({"user_input": user_input})
    return output["proteins"]

In [53]:
user_input = "Find proteins related to TP53"
proteins = query_proteins(user_input)
print("Relevant Proteins (ensp):", proteins)

Generated AQL Query: SELECT p.protein_name 
FROM Proteins p 
WHERE p.related_gene = 'TP53'
Error executing AQL query: [HTTP 400][ERR 1501] syntax error, unexpected identifier near 'SELECT p.protein_name 
FROM Prot...' at position 1:1
Relevant Proteins (ensp): []
